In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score

In [ ]:
iris = load_iris()
data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
target = pd.Series(iris.target, name='target')
df = pd.concat([data, target], axis=1)

print(df.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  


In [ ]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

print(f"Tamanho do conjunto de treino: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")

Tamanho do conjunto de treino: (75, 4)
Tamanho do conjunto de teste: (75, 4)


In [ ]:
# Criando o modelo LightGBM
model = lgb.LGBMClassifier(random_state=42)

# Treinando o modelo com os dados de treino
model.fit(X_train, y_train)

print("Modelo LightGBM treinado com sucesso!")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000028 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 71
[LightGBM] [Info] Number of data points in the train set: 75, number of used features: 4
[LightGBM] [Info] Start training from score -1.272966
[LightGBM] [Info] Start training from score -1.021651
[LightGBM] [Info] Start training from score -1.021651
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [ ]:
y_pred = model.predict(X_test)
print(f"Previsões no conjunto de teste: {y_pred}")

Previsões no conjunto de teste: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0 1 1 2 1 2 1 2 1 0 2 1 0 0 0 1 2 0 0 0 1 0 1 2 0 1 2 0 1 2
 1]


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo no conjunto de teste: {accuracy:.2f}")

Acurácia do modelo no conjunto de teste: 0.97


In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Carrega a base de dados Iris
iris = load_iris()
data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
target = pd.Series(iris.target, name='target')
df = pd.concat([data, target], axis=1)

# Vamos verificar os nomes das classes originais para usar na filtragem
print("Nomes das classes originais:", iris.target_names)

# Filtrando o DataFrame para manter apenas as amostras das classes 'setosa' e 'virginica'
# Para 'setosa', o target é 0. Para 'virginica', o target é 2.
df_filtered = df[df['target'].isin([0, 2])]

# Separa os dados filtrados em características (X) e alvo (y)
X = df_filtered.drop('target', axis=1)
y = df_filtered['target']

# Como agora temos apenas duas classes (0 e 2), para facilitar a interpretação
# em alguns contextos (como regressão logística binária, embora não seja o caso aqui),
# podemos remapear os rótulos para 0 e 1. Isso é opcional e não afeta o LightGBM.
# Se você quiser manter os rótulos originais (0 e 2), pode comentar as próximas duas linhas.
y = y.replace({0: 0, 2: 1})

# Divide os dados filtrados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Cria o modelo LightGBM (agora para um problema de classificação binária)
model = lgb.LGBMClassifier(random_state=42)

# Listas para armazenar as métricas de teste ao longo das iterações
evals_result = {}

# Treina o modelo com monitoramento no conjunto de teste
model.fit(X_train, y_train,
          eval_set=[(X_test, y_test)],
          eval_metric='binary_logloss', # Usamos 'binary_logloss' para classificação binária
          callbacks=[lgb.record_evaluation(evals_result)])

print("\nModelo LightGBM treinado com sucesso!")

# Faz previsões no conjunto de teste
y_pred = model.predict(X_test)
print(f"Previsões no conjunto de teste: {y_pred}")

# Avalia o desempenho do modelo (agora para classificação binária)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo no conjunto de teste: {accuracy:.2f}")

# Explora a importância dos recursos
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print("\nImportância dos recursos:")
print(feature_importances)

Nomes das classes originais: ['setosa' 'versicolor' 'virginica']
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 37, number of negative: 33
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000018 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 62
[LightGBM] [Info] Number of data points in the train set: 70, number of used features: 4
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.528571 -> initscore=0.114410
[LightGBM] [Info] Start training from score 0.114410
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 